# **IP3_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem.   """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [7]:
Markdown(response1.choices[0].message.content)

Parameters:

1. $N$: Total number of items that we need to pack.
2. $M$: Total number of boxes available.
3. $w_i$: Weight of the $i^{th}$ item where $1 \leq i \leq N$
4. $C$: Weight capacity of a box.

Decision Variables:

1. $x_{ij}$: This is a binary decision variable. $x_{ij} = 1$ if item $i$ is packed in box $j$ and $0$ otherwise, where $1 \leq i \leq N$ and $1 \leq j \leq M$.
2. $y_j$: This is a binary decision variable. $y_j = 1$ if box $j$ is used and $0$ otherwise, where $1 \leq j \leq M$.

Objective Function:

Our goal is to minimize the number of boxes used.
So, the objective function would be:
Minimize $\sum_{j=1}^{M} y_j$

Subject to:

1. Each item should be packed into one box: $\sum_{j=1}^{M} x_{ij} = 1$ for $1 \leq i \leq N$.
2. The total weight of items in each box should not exceed the capacity of that box: $\sum_{i=1}^{N} w_i \cdot x_{ij} \leq C \cdot y_j$ for $1 \leq j \leq M$.
3. If an item is packed in a box, that box is considered as used: $\sum_{i=1}^{N} x_{ij} \leq N \cdot y_j$ for $1 \leq j \leq M$.
4. Decision Variables: $x_{ij} \in \{0,1\}$ and $y_j \in \{0,1\}$ for $1 \leq i \leq N$ and $1 \leq j \leq M$.


### **Generate Pyomo Code**

In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [9]:
Markdown(response2.choices[0].message.content)

Below is a Python Pyomo code for specified optimization problem:

```python
from pyomo.environ import *

# Defining number of items and boxes (Sample Data)
N = 5
M = 3
weights = {1:2, 2:3, 3:5, 4:7, 5:9}  # Weight of each item (Sample Data)
C = 10  # Weight capacity of a box (Sample Data)

# Model
model = ConcreteModel()

# Sets
model.items = RangeSet(1,N)
model.boxes = RangeSet(1,M)

# Variables
model.x = Var(model.items, model.boxes, domain=Binary)
model.y = Var(model.boxes, domain=Binary)

# Objective
model.obj = Objective(expr = sum(model.y[j] for j in model.boxes), sense=minimize)

# Constraints
model.item_in_a_box = ConstraintList()
for i in model.items:
    model.item_in_a_box.add(sum(model.x[i,j] for j in model.boxes) == 1)

model.weight_limit = ConstraintList()
for j in model.boxes:
    model.weight_limit.add(sum(weights[i]*model.x[i,j] for i in model.items) <= C*model.y[j])

model.box_usage= ConstraintList()
for j in model.boxes:
    model.box_usage.add(sum(model.x[i,j] for i in model.items) <= N*model.y[j])

# Solve
SolverFactory('glpk').solve(model)

# Print the output
for j in model.boxes:
    if value(model.y[j]) == 1:
        print('Items in the box:', j, 'are:')
        for i in model.items:
            if value(model.x[i,j]) == 1:
                print('Item:', i)

```

Note: The user might need to adjust the solver based on its own installation. Here, 'glpk' is used for demonstrative purposes and might need adjustment according to the specific circumstance.

In the code above, N and M are the number of items and boxes, respectively. 'weights' stores the weight of each item and C is the weight capacity of a box. These are considered as sample data. You need to replace them with your actual values.

### **Run the code Generated by GPT4**

In [14]:
from pyomo.environ import *

# Defining number of items and boxes (Sample Data)
N = 5
M = 3
weights = {1:2, 2:3, 3:5, 4:7, 5:9}  # Weight of each item (Sample Data)
C = 10  # Weight capacity of a box (Sample Data)

# Model
model = ConcreteModel()

# Sets
model.items = RangeSet(1,N)
model.boxes = RangeSet(1,M)

# Variables
model.x = Var(model.items, model.boxes, domain=Binary)
model.y = Var(model.boxes, domain=Binary)

# Objective
model.obj = Objective(expr = sum(model.y[j] for j in model.boxes), sense=minimize)

# Constraints
model.item_in_a_box = ConstraintList()
for i in model.items:
    model.item_in_a_box.add(sum(model.x[i,j] for j in model.boxes) == 1)

model.weight_limit = ConstraintList()
for j in model.boxes:
    model.weight_limit.add(sum(weights[i]*model.x[i,j] for i in model.items) <= C*model.y[j])

model.box_usage= ConstraintList()
for j in model.boxes:
    model.box_usage.add(sum(model.x[i,j] for i in model.items) <= N*model.y[j])

# Solve
SolverFactory('glpk').solve(model)

# Print the output
for j in model.boxes:
    if value(model.y[j]) == 1:
        print('Items in the box:', j, 'are:')
        for i in model.items:
            if value(model.x[i,j]) == 1:
                print('Item:', i)

ValueError: Attempting to declare a block component using the name of a reserved attribute:
	items

In [15]:
from pyomo.environ import *

# Start - data inputted by human
N = 25
weights = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
C = 10  # Weight capacity of a box (Sample Data)
M = 15
#End
# Model
model = ConcreteModel()

# Sets
model.items = RangeSet(1,N)
model.boxes = RangeSet(1,M)

# Variables
model.x = Var(model.items, model.boxes, domain=Binary)
model.y = Var(model.boxes, domain=Binary)

# Objective
model.obj = Objective(expr = sum(model.y[j] for j in model.boxes), sense=minimize)

# Constraints
model.item_in_a_box = ConstraintList()
for i in model.items:
    model.item_in_a_box.add(sum(model.x[i,j] for j in model.boxes) == 1)

model.weight_limit = ConstraintList()
for j in model.boxes:
    model.weight_limit.add(sum(weights[i]*model.x[i,j] for i in model.items) <= C*model.y[j])

model.box_usage= ConstraintList()
for j in model.boxes:
    model.box_usage.add(sum(model.x[i,j] for i in model.items) <= N*model.y[j])

# Solve
SolverFactory('glpk').solve(model)

# Print the output
for j in model.boxes:
    if value(model.y[j]) == 1:
        print('Items in the box:', j, 'are:')
        for i in model.items:
            if value(model.x[i,j]) == 1:
                print('Item:', i)

ValueError: Attempting to declare a block component using the name of a reserved attribute:
	items

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [31]:
from pyomo.environ import *

# Start - data inputted by human
N = 24
weights = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
C = 9 # Weight capacity of a box (Sample Data)
M = 15
#End
# Model
model = ConcreteModel()

# Sets
model.item = RangeSet(1,N)
model.boxes = RangeSet(1,M)

# Variables
model.x = Var(model.item, model.boxes, domain=Binary)
model.y = Var(model.boxes, domain=Binary)

# Objective
model.obj = Objective(expr = sum(model.y[j] for j in model.boxes), sense=minimize)

# Constraints
model.item_in_a_box = ConstraintList()
for i in model.item:
    model.item_in_a_box.add(sum(model.x[i,j] for j in model.boxes) == 1)

model.weight_limit = ConstraintList()
for j in model.boxes:
    model.weight_limit.add(sum(weights[i]*model.x[i,j] for i in model.item) <= C*model.y[j])

model.box_usage= ConstraintList()
for j in model.boxes:
  model.box_usage.add(sum(model.x[i,j] for i in model.item) <= N*model.y[j])

# Solve
SolverFactory('glpk').solve(model)

# Print the output
for j in model.boxes:
    if value(model.y[j]) == 1:
        print('Items in the box:', j, 'are:')
        for i in model.item:
            if value(model.x[i,j]) == 1:
                print('Item:', i)
print(model.obj())

Items in the box: 1 are:
Item: 2
Item: 21
Items in the box: 2 are:
Item: 3
Item: 20
Items in the box: 3 are:
Item: 1
Item: 19
Items in the box: 4 are:
Item: 7
Item: 13
Items in the box: 5 are:
Item: 6
Item: 15
Items in the box: 6 are:
Item: 10
Item: 18
Items in the box: 7 are:
Item: 9
Item: 17
Items in the box: 8 are:
Item: 4
Item: 22
Items in the box: 9 are:
Item: 11
Item: 12
Items in the box: 10 are:
Item: 23
Items in the box: 11 are:
Item: 24
Items in the box: 14 are:
Item: 5
Item: 16
Items in the box: 15 are:
Item: 8
Item: 14
13.0
